In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import datasets, transformers, onnx, onnxruntime, torch, torch.nn as nn, numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

c:\source\repos\Models-Inferencing-On-Client-Sandbox\sandboxenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using BERT-base-uncased: https://huggingface.co/google-bert/bert-base-uncased

In [2]:
checkpoint = "google-bert/bert-base-uncased"

In [3]:
dataset = datasets.load_dataset("emotion")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

c:\source\repos\Models-Inferencing-On-Client-Sandbox\sandboxenv\Lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [5]:
print(len(full_train_dataset))
print(len(full_eval_dataset))

16000
2000


In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=6)
model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
metric = datasets.load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

C:\Users\garye\AppData\Local\Temp\ipykernel_25800\2330261002.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("accuracy")
c:\source\repos\Models-Inferencing-On-Client-Sandbox\sandboxenv\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [9]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments("test_trainer",
                                  per_device_train_batch_size=32,
                                  gradient_accumulation_steps=4,
                                  num_train_epochs=24, learning_rate=3e-05,
                                  evaluation_strategy="epoch",
                                  fp16=True)
trainer = Trainer(
    model = model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset
)

In [ ]:
print(full_train_dataset)
print(full_eval_dataset)

In [10]:
trainer.train()